In [2]:
import codecs
import pandas as pd
import itertools as it

from pandas import DataFrame
import os

In [3]:
stars_filepath = 'review_stars_rest_subset.txt'
review_txt_filepath = 'review_text_rest_subset.txt'

In [7]:
def line_review(filename):
    """
    generator function to read in reviews from the file
    and un-escape the original line breaks in the text
    """
    
    with codecs.open(filename, encoding='utf_8') as f:
        for review in f:
            yield review.replace('\\n', '\n')

In [6]:
def get_sample_review(review_number):
    """
    retrieve a particular review index
    from the reviews file and return it
    """
    
    return list(it.islice(line_review(review_txt_filepath),
                          review_number, review_number+1))[0]

In [5]:
def get_stars(review_number):
    """
    retrieve a particular review index
    from the reviews file and return it
    """
    
    return list(it.islice(line_review(stars_filepath),
                          review_number, review_number+1))[0]

In [8]:
get_stars(0).replace('\n', '')

'4'

In [9]:
get_sample_review(8)

"Came here for a burger as one of my friends said they had good burgers and people from work would come by twice a month for a burger run.  Ordered the original was was ok, wasn't as amazing as everyone says it to be. I think the veggie option would have been better.  A little pricy for a take-out burger. \r\n"

In [10]:
with open(stars_filepath) as f:
    stars = f.readlines()
# remove whitespace characters like `\n` at the end of each line
stars = [x.strip() for x in stars]

In [11]:
len(stars)

1570963

In [12]:
with open(review_txt_filepath) as f:
    texts = f.readlines()
# remove whitespace characters like `\n` at the end of each line
texts = [x.strip() for x in texts]

In [13]:
len(texts)

1570963

In [14]:
from collections import Counter
 
def balance_classes(xs, ys):
    """Undersample xs, ys to balance classes."""
    freqs = Counter(ys)
 
    #the least common class is the maximum number we want for all classes
    max_allowable = freqs.most_common()[-1][1]
    num_added = {clss: 0 for clss in freqs.keys()}
    new_ys = []
    new_xs = []
    for i, y in enumerate(ys):
        if num_added[y] < max_allowable:
            new_ys.append(y)
            new_xs.append(xs[i])
            num_added[y] += 1
    return new_xs, new_ys

In [15]:
print(Counter(stars))
balanced_x, balanced_y = balance_classes(texts, stars)
print(Counter(balanced_y))



Counter({'4': 546277, '5': 468813, '3': 302214, '2': 154033, '1': 99626})
Counter({'4': 99626, '3': 99626, '5': 99626, '1': 99626, '2': 99626})


In [16]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer

# This vectorizer breaks text into single words and bi-grams
# and then calculates the TF-IDF representation
vectorizer = TfidfVectorizer(ngram_range=(1,2))


# the 'fit' builds up the vocabulary from all the reviews
# while the 'transform' step turns each indivdual text into
# a matrix of numbers.
vectors = vectorizer.fit_transform(balanced_x)



/home/rlrognstad/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


CPU times: user 3min 34s, sys: 3.09 s, total: 3min 37s
Wall time: 3min 37s


In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(vectors, balanced_y, test_size=0.33, random_state=42)

In [21]:
%%time
from sklearn.svm import LinearSVC
 
# initialise the SVM classifier
classifier = LinearSVC()
 
# train the classifier
classifier.fit(X_train, y_train)


CPU times: user 1min 16s, sys: 336 ms, total: 1min 16s
Wall time: 1min 16s


In [24]:
preds = classifier.predict(X_test)
print(list(preds[:20]))
print(y_test[:20])
 


['2', '2', '1', '2', '5', '1', '4', '2', '2', '3', '2', '4', '3', '4', '5', '1', '4', '4', '3', '3']
['2', '2', '1', '2', '4', '1', '3', '2', '4', '2', '1', '5', '3', '4', '4', '2', '5', '4', '3', '3']


In [25]:
print(texts[0])
print(texts[8])

Who would have guess that you would be able to get fairly decent Vietnamese restaurant in East York?   Not quite the same as Chinatown in terms of pricing (slightly higher) but definitely one of the better Vietnamese restaurants outside of the neighbourhood. When I don't have time to go to Chinatown, this is the next best thing as it is down the street from me.  So far the only items I have tried are the phos (beef, chicken & vegetarian) - and they have not disappointed me! Especially the chicken pho.  Next time I go back, I'm going to try the banh cuon (steamed rice noodle) and the vermicelli!
Came here for a burger as one of my friends said they had good burgers and people from work would come by twice a month for a burger run.  Ordered the original was was ok, wasn't as amazing as everyone says it to be. I think the veggie option would have been better.  A little pricy for a take-out burger.


In [26]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, preds))

0.5919286057560696


In [27]:
from sklearn.metrics import classification_report
print(classification_report(y_test, preds))


             precision    recall  f1-score   support

          1       0.69      0.77      0.73     32656
          2       0.54      0.49      0.51     33054
          3       0.52      0.50      0.51     32829
          4       0.51      0.47      0.49     32732
          5       0.66      0.73      0.69     33112

avg / total       0.59      0.59      0.59    164383



In [29]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, preds))
 


[[25212  5966  1029   229   220]
 [ 8665 16033  6920  1071   365]
 [ 1761  6516 16574  6360  1618]
 [  364   940  5903 15430 10095]
 [  289   245  1173  7351 24054]]


Simplified (positive/negative) model

In [62]:
keep = set(['1','2','4','5'])
 
# calculate the indices for the examples we want to keep
keep_train_is = [i for i, y in enumerate(y_train) if y in keep]
keep_test_is = [i for i, y in enumerate(y_test) if y in keep]
 
# convert the train set
X_train2 = X_train[keep_train_is, :]
y_train2 = [y_train[i] for i in keep_train_is]
y_train2 = ["n" if (y == '1' or y == '2') else "p" for y in y_train2]
 
# convert the test set
X_test2 = X_test[keep_test_is, :]
y_test2 = [y_test[i] for i in keep_test_is]
y_test2 = ["n" if (y == '1' or y == '2') else "p" for y in y_test2]

In [58]:
y_test2[500]

'n'

In [65]:
print(X_train2[500])

  (0, 5315088)	0.037475675154455074
  (0, 5398464)	0.04018737537469805
  (0, 2177110)	0.05194824335153544
  (0, 4691650)	0.03699446493072189
  (0, 5440795)	0.010309474791763484
  (0, 508905)	0.011152616821063934
  (0, 4865008)	0.09191173220416929
  (0, 1978425)	0.026246506918178706
  (0, 2368335)	0.016699882869987824
  (0, 3187851)	0.05764214174214305
  (0, 4706767)	0.04872356155115173
  (0, 358176)	0.012162633923265902
  (0, 3235956)	0.015572073084245179
  (0, 774609)	0.017094901408819512
  (0, 1283826)	0.017817062816359246
  (0, 3312595)	0.012787856598331405
  (0, 5294885)	0.026935981274205094
  (0, 1417944)	0.015034040554364364
  (0, 4847001)	0.040177576918645416
  (0, 2019384)	0.01352158401655084
  (0, 4809344)	0.035470422564156334
  (0, 2474452)	0.05286326335276184
  (0, 1435004)	0.03639058627438363
  (0, 1903720)	0.013529352540620106
  (0, 2890112)	0.026684801184519394
  :	:
  (0, 70100)	0.06503580744215481
  (0, 516675)	0.07025830878461264
  (0, 1616413)	0.07420897853938688
  (0

In [60]:
%%time
# initialise the SVM classifier
classifier2 = LinearSVC()
classifier2.fit(X_train2, y_train2)
preds = classifier2.predict(X_test2)
print(classification_report(y_test2, preds))
print(confusion_matrix(y_test2, preds))

             precision    recall  f1-score   support

          n       0.96      0.96      0.96     65710
          p       0.96      0.96      0.96     65844

avg / total       0.96      0.96      0.96    131554

[[63059  2651]
 [ 2503 63341]]
CPU times: user 7.17 s, sys: 188 ms, total: 7.36 s
Wall time: 7.36 s


In [61]:
preds

array(['n', 'n', 'n', ..., 'n', 'p', 'p'], dtype='<U1')

pos/neg/neutral

In [87]:
# convert the train set
X_train3 = X_train
y_train3 = y_train

def conv_stars(stars):
    if (stars == '1' or stars == '2'):
        return "neg" 
    elif (stars == '3'):
        return "neut"
    else: return "pos"
        
y_train3 = [conv_stars(y) for y in y_train3]
    
    
# convert the test set
X_test3 = X_test
y_test3 = y_test2
y_test3= [conv_stars(y) for y in y_test3]

In [88]:
%%time
# initialise the SVM classifier
classifier3 = LinearSVC()
classifier3.fit(X_train3, y_train3)
preds = classifier3.predict(X_test3)
print(classification_report(y_test3, preds))
print(confusion_matrix(y_test3, preds))

ValueError: Found input variables with inconsistent numbers of samples: [131554, 164383]